In [1]:
import requests
import pandas as pd
import time

# Etherscan API key
ETHERSCAN_API_KEY = "2FFNJUA44X212BNX7J9DGSX63BDIZMBGIR"
WALLET_CSV = "Wallet id - Sheet1.csv"
OUTPUT_CSV = "compound_v2_transactions.csv"

# Compound V2 cToken Contracts
cToken_contracts = {
    "cDAI":  "0x5d3a536e4d6dbd6114cc1ead35777bab948e3643",
    "cUSDC": "0x39aa39c021dfbae8fac545936693ac917d5e7563",
    "cETH":  "0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5",
    "cWBTC": "0xccf4429db6322d5c611ee964527d42e5d685dd6a",
}
compound_contracts_set = set(cToken_contracts.values())  # For fast filtering

#  Load wallet addresses
wallet_df = pd.read_csv(WALLET_CSV)
wallets = wallet_df["wallet_id"].tolist()

#  Helper Functions
def get_normal_tx(address):
    """Query Etherscan for normal transactions of a wallet"""
    url = f"https://api.etherscan.io/api"
    params = {
        "module": "account",
        "action": "txlist",
        "address": address,
        "startblock": 0,
        "endblock": 99999999,
        "page": 1,
        "offset": 10000,
        "sort": "asc",
        "apikey": ETHERSCAN_API_KEY
    }
    try:
        response = requests.get(url, params=params)
        data = response.json()
        if data["status"] == "1":
            return data["result"]
        elif data["status"] == "0" and data["message"] == "No transactions found":
            return []
        else:
            print(f"[Normal Tx] Etherscan Error for {address}: {data}")
            return []
    except Exception as e:
        print(f"[Normal Tx] Error: {e}")
        return []

def get_internal_tx(address):
    """Query Etherscan for internal transactions of a wallet"""
    url = f"https://api.etherscan.io/api"
    params = {
        "module": "account",
        "action": "txlistinternal",
        "address": address,
        "startblock": 0,
        "endblock": 99999999,
        "page": 1,
        "offset": 10000,
        "sort": "asc",
        "apikey": ETHERSCAN_API_KEY
    }
    try:
        response = requests.get(url, params=params)
        data = response.json()
        if data["status"] == "1":
            return data["result"]
        elif data["status"] == "0" and data["message"] == "No transactions found":
            return []
        else:
            print(f"[Internal Tx] Etherscan Error for {address}: {data}")
            return []
    except Exception as e:
        print(f"[Internal Tx] Error: {e}")
        return []

def filter_compound_related(txs):
    """Filter txs involving Compound V2 contracts"""
    return [tx for tx in txs if tx.get('to') and tx['to'].lower() in compound_contracts_set]

#  Main Loop
all_data = []

for wallet in wallets:
    print(f" Fetching transactions for wallet: {wallet}")
    try:
        normal_txs = get_normal_tx(wallet)
        internal_txs = get_internal_tx(wallet)

        combined = filter_compound_related(normal_txs + internal_txs)

        for tx in combined:
            tx['wallet'] = wallet
            tx['contract_name'] = next(
                (name for name, addr in cToken_contracts.items() if addr == tx['to'].lower()),
                "Unknown"
            )
            all_data.append(tx)

        time.sleep(0.25)  # To stay under Etherscan rate limit 5 req/sec
    except Exception as e:
        print(f"Error processing wallet {wallet}: {e}")

#  Save to CSV
if all_data:
    df = pd.DataFrame(all_data)
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"\n Saved {len(df)} Compound-related transactions to {OUTPUT_CSV}")
else:
    print("\n No Compound-related transactions found.")


 Fetching transactions for wallet: 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
 Fetching transactions for wallet: 0x06b51c6882b27cb05e712185531c1f74996dd988
 Fetching transactions for wallet: 0x0795732aacc448030ef374374eaae57d2965c16c
 Fetching transactions for wallet: 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
 Fetching transactions for wallet: 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
 Fetching transactions for wallet: 0x104ae61d8d487ad689969a17807ddc338b445416
 Fetching transactions for wallet: 0x111c7208a7e2af345d36b6d4aace8740d61a3078
 Fetching transactions for wallet: 0x124853fecb522c57d9bd5c21231058696ca6d596
 Fetching transactions for wallet: 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc
 Fetching transactions for wallet: 0x1656f1886c5ab634ac19568cd571bc72f385fdf7
 Fetching transactions for wallet: 0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22
 Fetching transactions for wallet: 0x19df3e87f73c4aaf4809295561465b993e102668
 Fetching transactions for wallet: 0x1ab2ccad4fc97c9968ea87d4435

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# STEP 1: Load and preprocess transaction data
df = pd.read_csv("compound_v2_transactions.csv")

# Convert types and normalize
df['value'] = pd.to_numeric(df['value'], errors='coerce') / 1e18         
df['gasPrice'] = pd.to_numeric(df['gasPrice'], errors='coerce') / 1e9    

# Lowercase consistency
df['wallet'] = df['wallet'].str.lower()
df['from'] = df['from'].str.lower()
df['to'] = df['to'].str.lower()

# Fill NaNs
df['value'] = df['value'].fillna(0)
df['gasPrice'] = df['gasPrice'].fillna(0)

# Add feature: self-transfer
df['is_self_transfer'] = (df['from'] == df['to']).astype(int)

# === STEP 2: Aggregate per wallet
def total_sent(group):
    return group.loc[group['from'] == group['wallet'], 'value'].sum()

def total_received(group):
    return group.loc[group['to'] == group['wallet'], 'value'].sum()

wallet_features = df.groupby('wallet').agg(
    tx_count=('hash', 'nunique'),
    unique_contracts=('to', 'nunique'),
    total_value_sent=('value', lambda x: total_sent(df.loc[x.index])),
    total_value_received=('value', lambda x: total_received(df.loc[x.index])),
    avg_gas_price=('gasPrice', 'mean'),
    num_self_transfers=('is_self_transfer', 'sum')
).reset_index()

# Derived features
wallet_features['net_value'] = wallet_features['total_value_received'] - wallet_features['total_value_sent']
wallet_features['send_receive_ratio'] = wallet_features['total_value_sent'] / (wallet_features['total_value_received'] + 1e-9)
wallet_features['activity_score'] = wallet_features['tx_count'] + wallet_features['unique_contracts']

# STEP 3: Normalize features for scoring
scaler = MinMaxScaler()

normalized_data = scaler.fit_transform(pd.DataFrame({
    'tx_count': wallet_features['tx_count'],
    'avg_gas_price': wallet_features['avg_gas_price'],
    'send_receive_ratio': wallet_features['send_receive_ratio'],
    'net_outflow': -wallet_features['net_value'], 
    'num_self_transfers': wallet_features['num_self_transfers']
}))

wallet_features[['norm_tx_count', 'norm_avg_gas', 'norm_sr_ratio', 'norm_net_outflow', 'norm_self_transfers']] = normalized_data

# STEP 4: Risk Scoring 
# Define weights for each feature
weights = {
    'norm_tx_count': 0.15,
    'norm_avg_gas': 0.20,
    'norm_sr_ratio': 0.25,
    'norm_net_outflow': 0.30,
    'norm_self_transfers': 0.10
}

wallet_features['raw_risk'] = (
    weights['norm_tx_count'] * wallet_features['norm_tx_count'] +
    weights['norm_avg_gas'] * wallet_features['norm_avg_gas'] +
    weights['norm_sr_ratio'] * wallet_features['norm_sr_ratio'] +
    weights['norm_net_outflow'] * wallet_features['norm_net_outflow'] +
    weights['norm_self_transfers'] * wallet_features['norm_self_transfers']
)

wallet_features['risk_score'] = (wallet_features['raw_risk'] * 1000).astype(int)

# STEP 5: Output final result 
wallet_features[['wallet', 'risk_score']].sort_values(by='risk_score', ascending=False)\
    .to_csv("wallet_risk_scores.csv", index=False)

print(" Risk scores saved to wallet_risk_scores.csv")


 Risk scores saved to wallet_risk_scores.csv
